In [13]:
import random
import spacy
from spacy.training.example import Example
import pandas as pd

In [14]:
nlp = spacy.blank("en")

# Add the custom component to the pipeline
parser = nlp.add_pipe("parser")

TRAIN_DATA = [
    ("Find a cafe with few people", {"heads": [0, 2, 0, 5, 5, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    ("Find a hotel with many rooms near the beach", {"heads": [0, 2, 0, 5, 5, 2, 7, 7, 7], "deps": ["ROOT", "-", "PLACE","-","QUANTITY","ATTRIBUTE", "-", "-", "LOCATION"]}),
    ("I need a room with enough space", {"heads": [1, 1, 3, 1, 6, 6, 3], "deps": ["-", "ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    
    ("Show me the store that sells some unique products", {"heads": [0, 0, 3, 0, 5, 3, 8, 8, 3], "deps": ["ROOT", "-", "-", "PLACE", "-", "-", "QUANTITY", "-", "ATTRIBUTE"]}),
    ("Find a restaurant with all the menu items", {"heads": [0, 2, 0, 6, 7, 6, 7, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "-", "PRODUCT", "ATTRIBUTE"]}),
    ("Show me the hostel with half the rooms decorated" , {"heads": [0, 0, 3, 0, 5, 7, 7, 3, 7], "deps": ["ROOT", "-", "-", "PLACE", "-", "QUANTITY", "-", "ATTRIBUTE", "-"]}),
    
    ("Find a restaurant with whole cakes", {"heads": [0, 2, 0, 5, 5, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    ("Show me the shop with enough supplies", {"heads": [0, 0, 3, 0, 6, 6, 3], "deps": ["ROOT", "-", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
    ("Visit a library with numerous books", {"heads": [0, 2, 0, 5, 5, 2], "deps": ["ROOT", "-", "PLACE", "-", "QUANTITY", "ATTRIBUTE"]}),
]



for text, annotations in TRAIN_DATA:
    for dep in annotations.get('deps', []):
        parser.add_label(dep)

n_iter = 25

# Training loop with multiple iterations
optimizer = nlp.begin_training()
for iteration in range(n_iter):
    # Shuffle the training data
    random.shuffle(TRAIN_DATA)
    losses = {}

    for text, annotations in TRAIN_DATA:
        # Create an Example object
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # Update the model
        nlp.update([example],drop=0.25,losses=losses)

# After training, you can test the model
tests = [
    "find a hotel with many rooms near the beach",
    "show me the closest gym that's open late with enough space",
    "show me the store that sells some unique products",
    "find a restaurant with all the menu items",
    "show me the hostel with half the rooms decorated",
    "find a restaurant with whole cakes",
    "show me the shop with enough supplies near work",
    "visit a library with numerous books",
]

docs = nlp.pipe(tests)
for doc in docs:
    # print(doc.text)
    # print([(t.text, t.dep_, t.head.text) for t in doc if t.dep_ != "-"])
    # print()
    df = pd.DataFrame([(t.text, t.dep_, t.head.text) for t in doc if t.dep_ != "-"], columns=['text', 'dep', 'head'])
    print(df)
    print()

    text       dep   head
0   find      ROOT   find
1  hotel     PLACE   find
2   many  QUANTITY  rooms
3    the      ROOT    the
4  beach  LOCATION    the

      text        dep     head
0     show       ROOT     show
1  closest      PLACE     show
2     late      PLACE  closest
3   enough   QUANTITY    space
4    space  ATTRIBUTE     late

    text       dep      head
0   show      ROOT      show
1  store     PLACE      show
2   some  QUANTITY  products

